In [ ]:
import os
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt
import re


In [ ]:
# Load the sample data from ./sample_data/
data_dir = "./sample_data/"
data_files = [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f))]
data_files

In [ ]:
raw_contents = [] # list of lists (agent name, data string)
for data_file in data_files:
    f = open(data_file, mode="r")
    contents = f.read()
    agent_name = data_file[data_file.find("a/")+2:-4]
    raw_contents.append([agent_name, contents])
raw_contents

In [ ]:
# Regex match
parsed_data = {} # key = agent name, value = parsed data
for raw_content in raw_contents:
    agent_name, lines = raw_content
    lines = lines.split("\n")
    agent_data = {}
    curr_bomb_count = -1
    for line in lines:
        # extract bomb count #
        matches = re.findall(r"BOMB_COUNT = (\d+)", line)
        if len(matches) != 0:
            curr_bomb_count = int(matches[0])
            agent_data[curr_bomb_count] = []
            continue
        
        
        # extract score sample
        matches = re.findall(r".+?(?=bomb_count)bomb_count = (\d+), score = (\d+)", line)
        if len(matches) > 0:
            max_score, actual_score = matches[0]
            agent_data[curr_bomb_count].append(int(actual_score))
                
    # add agent data to parsed data
    parsed_data[agent_name] = agent_data

parsed_data

In [ ]:
# now that we have the actual scores for each bomb_count for each agent, we need to convert make a dict,
# matching agent name to a list of (total_score / total_max_score)
score_percentages = {}
for agent_name in parsed_data.keys():
    curr_percentages = []
    for max_score in parsed_data[agent_name].keys():
        if max_score == 0:
            curr_percentages.append(1.0)
        else:
            scores = parsed_data[agent_name][max_score]
            total_score = sum(scores)
            total_max_score = max_score * len(scores)
            curr_percentages.append(total_score / total_max_score)
    score_percentages[agent_name] = curr_percentages

score_percentages

In [ ]:
# Plot the data.
plt.figure(figsize=(20,10))
plt.rcParams["font.size"] = "20"
plt.xlabel("Bomb Density")
plt.ylabel("Average Score / Max Score")
for name, y_axis in score_percentages.items():
    densities = [x / max(parsed_data[agent_name].keys()) for x in parsed_data[agent_name].keys()]
    plt.plot(densities, y_axis, label=name)

plt.title("Bomb Density vs Score")
plt.legend()